In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from certa import local_explain, triangles_method

from certa.utils import merge_sources
from eval_emt import get_original_prediction
from models.bert import EMTERModel

In [10]:
model = EMTERModel()
model = model.load('../models/bert/abt_buy')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [11]:
def predict_fn(x, model, ignore_columns=['ltable_id', 'rtable_id', 'label']):
    return model.predict(x)

In [12]:
datadir = '../datasets/abt_buy'
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
valid = pd.read_csv(datadir + '/valid.csv')
test = pd.read_csv(datadir + '/test.csv')

In [ ]:
test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], []).dropna()

In [33]:
rand_row = test_df.iloc[27]
l_id = int(rand_row['ltable_id'])
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]


In [34]:
l_tuple

id                                                           572
name           samsung 46 ' series 5 lcd black flat panel hdt...
description    samsung 46 ' series 5 lcd black flat panel hdt...
price                                                        NaN
Name: 572, dtype: object

In [35]:
r_tuple

id                                                           695
name                       samsung 4 series ln22a450 22 ' lcd tv
description    samsung ln22a450 22 ' widescreen 720p lcd hdtv...
price                                                     397.77
Name: 695, dtype: object

In [41]:
# false negative
predicted_class = np.argmax(get_original_prediction(l_tuple, r_tuple, model))
predicted_class

We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 


Test: 100%|██████████| 1/1 [00:00<00:00,  4.36it/s]
/Users/tommasoteofili/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0

In [42]:
tmin = 0.5
tmax = 0.5
local_samples, gleft_df, gright_df = local_explain.dataset_local(l_tuple, r_tuple, model, lsource, rsource, datadir,
                                                                            tmin, tmax, predict_fn, num_triangles=100,
                                                                            class_to_explain=predicted_class, use_predict=True,
                                                                            max_predict=500)

We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 


Test: 100%|██████████| 1/1 [00:00<00:00,  4.35it/s]
/Users/tommasoteofili/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 


Test: 100%|██████████| 63/63 [01:45<00:00,  1.68s/it]


We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 


Test: 100%|██████████| 63/63 [01:44<00:00,  1.66s/it]


We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 


Test: 100%|██████████| 21/21 [00:33<00:00,  1.59s/it]

+perturbed neighborhood: 157



/Users/tommasoteofili/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
explanation, flipped_pred, triangles = triangles_method.explainSamples(local_samples,
                                                      [pd.concat([lsource, gright_df]),
                                                       pd.concat([rsource, gleft_df])],
                                                      model, predict_fn, predicted_class,
                                                      2, False, discard_bad=False)
explanation

  0%|          | 0/100 [00:00<?, ?it/s]

We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 



Test: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]

We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 



/Users/tommasoteofili/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  1%|          | 1/100 [00:03<06:23,  3.87s/it]

...


Test: 100%|██████████| 2/2 [00:02<00:00,  1.27s/it]

We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 




 65%|██████▌   | 65/100 [04:19<02:19,  3.98s/it]

We use the device: 'cpu' and 0 gpu's. Important: distributed and 16-bits training is currently not implemented! 



Test:  50%|█████     | 1/2 [00:01<00:01,  1.71s/it]